In [1]:
import base64
import hmac
import time
import requests.auth
import requests
import pandas as pd

class BudaHMACAuth(requests.auth.AuthBase):
    """Adjunta la autenticación HMAC de Buda al objeto Request."""

    def __init__(self, api_key: str, secret: str):
        self.api_key = api_key
        self.secret = secret

    def get_nonce(self) -> str:
        # 1. Generar un nonce (timestamp en microsegundos)
        return str(int(time.time() * 1e6))

    def sign(self, r, nonce: str) -> str:
        # 2. Preparar string para firmar
        components = [r.method, r.path_url]
        if r.body:
            encoded_body = base64.b64encode(r.body).decode()
            components.append(encoded_body)
        components.append(nonce)
        msg = ' '.join(components)
        # 3. Obtener la firma
        h = hmac.new(key=self.secret.encode(),
                        msg=msg.encode(),
                        digestmod='sha384')
        signature = h.hexdigest()
        return signature

    def __call__(self, r):
        nonce = self.get_nonce()
        signature = self.sign(r, nonce)
        # 4. Adjuntar API-KEY, nonce y firma al header del request
        r.headers['X-SBTC-APIKEY'] = self.api_key
        r.headers['X-SBTC-NONCE'] = nonce
        r.headers['X-SBTC-SIGNATURE'] = signature
        return r
        
def last_price(market_id, fiat='clp'):
    market_id = market_id.lower() + '-' + fiat.lower()
    url = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
    response = requests.get(url)
    ticker_info = response.json()
    return float(ticker_info['ticker']['last_price'][0])
# Para autenticar una llamada se debe incluir `auth` en el request
api_key = ''
secret = ''

In [2]:
url = f'https://www.buda.com/api/v2/balances'
auth = BudaHMACAuth(api_key, secret)
response = requests.get(url, auth=auth)
balances = response.json()['balances']

In [3]:
portfolio = []
for x in balances:
    portfolio.append(x['amount'])
    
df = pd.DataFrame(portfolio, columns=['Cantidad', 'Activo'], dtype=float)
# esto funcionara solo si tenemos una moneda FIAT
fiat = 'CLP' # modificar si usas otra fiat
df = df[df['Activo']!=fiat] # Saco Fiat Currency del portfolio
df['Precio'] = df['Activo'].apply(last_price)
df['Pos'] = df['Cantidad'] * df['Precio']

In [6]:
# Detalle de la cartera valoriza en FIAT (columna Pos)
df

,Cantidad,Activo,Precio,Pos
0,0.722332,BCH,498529.00,3.601032e+05
1,0.012806,BTC,50500000.00,6.467202e+05
3,0.378828,ETH,3648344.00,1.382094e+06
4,2.573590,LTC,162403.00,4.179588e+05
5,163.650430,USDC,815.01,1.333767e+05


In [7]:
# Utilizo precio de la API de kraken,
# Desarrollar un forma de obtener datos historicos con la API de Buda.com??
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)
# interval: {1, 5, 15, 30, 60, 240, 1440, 10080, 21600}..

data_hist = {}
# Realizo multiples llamadas para obtener el historico de cada activo
for i in df['Activo']:
    coin= i+'USD'
    data_hist[i] = k.get_ohlc_data(coin, interval=240, ascending = True)[0][['close']].rename(columns={'close':i})

public call frequency exceeded (seconds=0.463359) 
 sleeping for 5 seconds
public call frequency exceeded (seconds=0.425006) 
 sleeping for 5 seconds


In [8]:
import numpy as np
def maxdowndraw(df):
    df = np.log(df) - np.log(df.shift(1))
    mini = df.min()[0]
    mxdd2 = (df.shift(1) + df).min()[0]
    mxdd3 = (df.shift(2) + df.shift(1) + df).min()[0]
    mxdd4 = (df.shift(3) +df.shift(2) + df.shift(1) + df).min()[0]
    mxdd5 = (df.shift(4) +df.shift(3) +df.shift(2) + df.shift(1) + df).min()[0]
    # mxdd = min([mini, mxdd2, mxdd3, mxdd4, mxdd5])
    return min([mini, mxdd2, mxdd3, mxdd4, mxdd5])

def cvar(df, q=0.02):
    df = df.pct_change()
    df = df[df[df.columns[0]]<= df.quantile(q)[0]] 
    return df.mean()[0]


VAR is the maximum loss over a target horizon such that there is a low, prespec- ified probability that the actual loss will be larger.

In [9]:
# Medidas de Riesgo Estaticas

# Desviacion Estandar del activo
df['std'] = df['Activo'].apply(lambda x: data_hist[x].pct_change().std()[0])

# Var99% de cada activo
df['VaR(99%)'] = df['Activo'].apply(lambda x: data_hist[x].pct_change().quantile(0.01)[0])

# promedio del 2% de los peores retornos
df['CVaR(98%)'] = df['Activo'].apply(lambda x: cvar(data_hist[x]))

# Peor Retorno
df['Ret_min'] = df['Activo'].apply(lambda x: data_hist[x].pct_change().min()[0])

# Peor Retorno acumulado en 5 periodos
df['MaxDD5'] = df['Activo'].apply(lambda x: maxdowndraw(data_hist[x]))

In [12]:
df_ret = pd.concat(data_hist, axis = 1).pct_change().dropna()
df_ret.columns = [(col[0]) for col in df_ret.columns.values]
# Retornos * Posicion
df_ret = df_ret.mul(df['Pos'].values)

In [14]:
# resumen metricas de riesgo resultado porcentual, para obtenerlo en pesos multiplicar Pos por cada metrica de riesgo
df

,Cantidad,Activo,Precio,Pos,std,VaR(99%),CVaR(98%),Ret_min,MaxDD5
0,0.722332,BCH,498529.00,3.601032e+05,0.017352,-0.042063,-0.049146,-0.098887,-0.187149
1,0.012806,BTC,50500000.00,6.467202e+05,0.014478,-0.033107,-0.039731,-0.082624,-0.122455
3,0.378828,ETH,3648344.00,1.382094e+06,0.017878,-0.042415,-0.047461,-0.083790,-0.140865
4,2.573590,LTC,162403.00,4.179588e+05,0.019147,-0.046646,-0.053501,-0.130510,-0.212889
5,163.650430,USDC,815.01,1.333767e+05,0.000083,-0.000200,-0.000200,-0.000200,-0.000200


In [16]:
import plotly.graph_objects as go
labels = df_ret.columns.values.tolist()
values = [x * -1 for x in df_ret.quantile(0.01).values.tolist() ]

fig = go.Figure(data=[go.Pie(labels=labels, values=values)], layout=go.Layout( title=go.layout.Title(text="Riesgo Distribuido por Activo")))
fig.show()